In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_data(file_name):
    df = pd.read_csv("D:/大创/Data/" + file_name ,encoding='gbk')
    columns = df.columns
    df.fillna(df.mean(numeric_only=True) ,inplace=True)
    return df

In [2]:
df = load_data('bread_flow.csv')
df

,year,month,day,period_day,week,weekday_weekend,number
0,2016,10,30,morning,星期日,weekend,86
1,2016,10,30,afternoon,星期日,weekend,84
2,2016,10,31,morning,星期一,weekday,103
3,2016,10,31,afternoon,星期一,weekday,88
4,2016,10,31,evening,星期一,weekday,8
...,...,...,...,...,...,...,...
410,2017,4,8,morning,星期六,weekend,94
411,2017,4,8,afternoon,星期六,weekend,103
412,2017,4,8,night,星期六,weekend,8
413,2017,4,9,morning,星期日,weekend,32


In [3]:
df['period_day'].value_counts()

period_day
morning      159
afternoon    158
evening       94
night          4
Name: count, dtype: int64

In [4]:
week = []

for w in df['week']:
    if w == '星期一':
        week.append(1)
    if w == '星期二':
        week.append(2)
    if w == '星期三':
        week.append(3)
    if w == '星期四':
        week.append(4)
    if w == '星期五':
        week.append(5)
    if w == '星期六':
        week.append(6)
    if w == '星期日':
        week.append(7)
        
df['week'] = week

In [5]:
df = df[2:]
df = df[:-2]
df = df.reset_index(drop=True)
df

,year,month,day,period_day,week,weekday_weekend,number
0,2016,10,31,morning,1,weekday,103
1,2016,10,31,afternoon,1,weekday,88
2,2016,10,31,evening,1,weekday,8
3,2016,11,1,morning,2,weekday,56
4,2016,11,1,afternoon,2,weekday,92
...,...,...,...,...,...,...,...
406,2017,4,7,afternoon,5,weekday,57
407,2017,4,7,evening,5,weekday,2
408,2017,4,8,morning,6,weekend,94
409,2017,4,8,afternoon,6,weekend,103


In [6]:
def complete(df):
# 所有可能的时段列表
    all_periods = ['morning', 'afternoon', 'evening', 'night']

    # 创建空的DataFrame作为结果
    result = pd.DataFrame(columns=df.columns)

    # 遍历每一天
    for _, group in df.groupby(['year', 'month', 'day']):
        day_data = group.copy()  # 当天的数据
        existing_periods = day_data['period_day'].unique()  # 当天存在的时段

        # 检查是否存在四个时段的数据
        if set(all_periods) == set(existing_periods):
            result = pd.concat([result, day_data])  # 直接复制到结果DataFrame中
        else:
            #print(existing_periods)
            # 缺失时段的数据
            missing_periods = [period for period in all_periods if period not in existing_periods]
            result = pd.concat([result, day_data])

            for missing_period in missing_periods:
                new_row = {
                    'year': day_data.iloc[0]['year'],
                    'month' : day_data.iloc[0]['month'],
                    'day' : day_data.iloc[0]['day'],
                    'period_day' : missing_period,
                    'week' : day_data.iloc[0]['week'],
                    'weekday_weekend' : day_data.iloc[0]['weekday_weekend'],
                    'number' : 0
                }
                result.loc[len(result)] = new_row
    return result
    

# 输出补全后的结果
result = complete(df)
result

,year,month,day,period_day,week,weekday_weekend,number
0,2016,10,31,morning,1,weekday,103
1,2016,10,31,afternoon,1,weekday,88
2,2016,10,31,evening,1,weekday,8
3,2016,10,31,night,1,weekday,0
3,2016,11,1,morning,2,weekday,56
...,...,...,...,...,...,...,...
623,2017,4,7,night,5,weekday,0
408,2017,4,8,morning,6,weekend,94
409,2017,4,8,afternoon,6,weekend,103
410,2017,4,8,night,6,weekend,8


In [7]:
# 假设您的DataFrame名为df
result['date'] = pd.to_datetime(result[['year', 'month', 'day']])

# 定义 period_day 的排序顺序
period_day_order = ['morning', 'afternoon', 'evening', 'night']

# 将 period_day 转换为 Categorical 类型，并指定排序顺序
result['period_day'] = pd.Categorical(result['period_day'], categories=period_day_order, ordered=True)

# 按照日期（date）和 period_day 进行排序
result_sorted = result.sort_values(by=['date', 'period_day'])

# 打印排序后的结果
result_sorted

,year,month,day,period_day,week,weekday_weekend,number,date
0,2016,10,31,morning,1,weekday,103,2016-10-31
1,2016,10,31,afternoon,1,weekday,88,2016-10-31
2,2016,10,31,evening,1,weekday,8,2016-10-31
3,2016,10,31,night,1,weekday,0,2016-10-31
3,2016,11,1,morning,2,weekday,56,2016-11-01
...,...,...,...,...,...,...,...,...
623,2017,4,7,night,5,weekday,0,2017-04-07
408,2017,4,8,morning,6,weekend,94,2017-04-08
409,2017,4,8,afternoon,6,weekend,103,2017-04-08
627,2017,4,8,evening,6,weekend,0,2017-04-08


In [10]:
result.to_csv('D:/大创/temp/bread.csv',sep= ',', encoding='gbk',index=False)